In [ ]:
import gglasso
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
import os
import matplotlib.pyplot as plt

from numpy import genfromtxt

from numpy.linalg import matrix_rank
from matplotlib.pyplot import figure
from scipy import stats
from scipy.linalg import eigh
from numpy import genfromtxt
from datetime import datetime

from gglasso.solver.admm_solver import ADMM_MGL
from gglasso.problem import glasso_problem

from gglasso.helper.data_generation import generate_precision_matrix, group_power_network, sample_covariance_matrix
from gglasso.helper.basic_linalg import adjacency_matrix
from gglasso.helper.data_generation import time_varying_power_network, sample_covariance_matrix
from gglasso.helper.experiment_helper import lambda_grid, discovery_rate, error
from gglasso.helper.utils import get_K_identity
from gglasso.helper.experiment_helper import plot_evolution, plot_deviation, surface_plot, single_heatmap_animation
from gglasso.helper.model_selection import aic, ebic, K_single_grid

In [ ]:
def calculate_edge_probablity(data=list, indices=list):
    
    P = np.zeros((436, 436))
    
    for i in indices:
        x = data[i, :]
        x = (x != 0).astype(int)

        P = P + x
    
    P = P / len(indices)
    
    return pd.DataFrame(P)

### Read data

#### Remove outliers

The outliers are found after manual checking of SGL solution for all samples.

In [ ]:
corr_all_ix = np.arange(0, 950)
outliers_ix = [96, 144, 210, 522]

corr_filtered_ix = np.array([i for i in corr_all_ix if i not in outliers_ix])
corr_filtered_ix.shape

In [ ]:
storage_dir = "/lustre/groups/bds01/datasets/brains/"

In [ ]:
outliers = []

for i in outliers_ix:
    outliers.append(genfromtxt(storage_dir + "corr_matrices/corr{0}.csv".format(i), delimiter=','))

    
outliers = np.array(outliers)
outliers.shape

In [ ]:
fig, axes = plt.subplots(2, 2, sharex=True, figsize=(30,30))

ax = sns.heatmap(outliers[0, :], ax=axes[0][0], center=0, vmin = -0.5, vmax = 0.5, 
                 square = True, cbar = False, cmap = "coolwarm", xticklabels=False, yticklabels=False)
ax.set_title('outlier_{0}'.format(outliers_ix[0]))

ax = sns.heatmap(outliers[1, :], ax=axes[0][1], center=0, vmin = -0.5, vmax = 0.5,
                 square = True, cbar = False, cmap = "coolwarm", xticklabels=False, yticklabels=False)
ax.set_title('outlier_{0}'.format(outliers_ix[1]))

ax = sns.heatmap(outliers[2, :], ax=axes[1][0], center=0, vmin = -0.5, vmax = 0.5,
                 square = True, cbar = False, cmap = "coolwarm", xticklabels=False, yticklabels=False)
ax.set_title('outlier_{0}'.format(outliers_ix[2]))

ax = sns.heatmap(outliers[3, :], ax=axes[1][1], center=0, vmin = -0.5, vmax = 0.5,
                 square = True, cbar = False, cmap = "coolwarm", xticklabels=False, yticklabels=False)
ax.set_title('outlier_{0}'.format(outliers_ix[3]))
    
fig.savefig("../../plots/outliers_heatmap.png")

### HMGU cluster

In [ ]:
### on cloud
corr = []

for i in corr_filtered_ix[:10]:
    corr.append(genfromtxt(storage_dir + "corr_matrices/corr{0}.csv".format(i), delimiter=','))

    
corr = np.array(corr)
corr.shape

### On premisis

In [ ]:
start = 0
stop = 50

sub_corr = []

### on premisis
for i in range(start, stop):
    sub_corr.append(genfromtxt("../../data/sub_corr50/sub_corr{0}.csv".format(i), delimiter=','))

    
sub_corr = np.array(sub_corr)
sub_corr.shape

### Single GL

### On premises

In [ ]:
N = [sub_corr.shape[1]]*sub_corr.shape[0]
lambda1_range = np.logspace(0, -3, 8)
print("Lambda range: {0}".format(lambda1_range))

est_uniform, est_indv, statistics = K_single_grid(sub_corr, lambda1_range, N, 
                                                  method = 'eBIC', gamma = 0.3, 
                                                  latent = False, use_block = True)


K = "50"

if not os.path.exists("../data/est_uniform{0}/".format(K)):
    os.mkdir("../data/est_uniform{0}/".format(K))
    
if not os.path.exists("../data/est_individ{0}/".format(K)):
    os.mkdir("../data/est_individ{0}/".format(K))

# dump matrices into csv
for i in range(start, stop):
    np.savetxt("../data/est_uniform{0}/est_uniform{1}.csv".format(K, i), est_uniform["Theta"][i], 
               delimiter=",", header='')
    np.savetxt("../data/est_individ{0}/est_individ{1}.csv".format(K, i), est_indv["Theta"][i], 
               delimiter=",", header='')
    
with open("../data/statistics{0}.txt".format(K), 'w') as f:
    print(statistics, file=f)

### HMGU cluster

In [ ]:
lambda1_range = np.logspace(-0.9, -1.5, 10)
lambda1_range

In [ ]:
K = len(corr)
N = K*[corr.shape[1]]

In [ ]:
start_time = datetime.now()

est_uniform, est_indv, statistics = K_single_grid(corr[:2, :], lambda1_range, N[:2], 
                                                  method = 'eBIC', gamma = 0.3, 
                                                  latent = False, use_block = True)


end_time = datetime.now()

run_time = end_time - start_time

statistics['time'] = run_time
print("--- TIME: {0} ---".format(run_time))

In [ ]:
with open('statistics_SGL_0_300.txt', 'w') as f:
    print(statistics, file=f)

In [ ]:
K = est_uniform["Theta"].shape[0]

In [ ]:
if not os.path.exists(storage_dir + "/est_uniform_test/"):
    os.makedirs(storage_dir + "/est_uniform_test/")
    
if not os.path.exists(storage_dir + "/est_individ_test/"):
    os.makedirs(storage_dir + "/est_individ_test/")

In [ ]:
# dump matrices into csv
for i in corr_filtered_ix[:2]:
    np.savetxt(storage_dir + "/est_uniform_test/est_uniform{0}.csv".format(i), est_uniform["Theta"][i], 
               delimiter=",", header='')
    np.savetxt(storage_dir + "/est_individ_test/est_individ{0}.csv".format(i), est_indv["Theta"][i], 
               delimiter=",", header='')

In [ ]:
# !jupyter nbconvert --to script --no-prompt SGL.ipynb

### Read solution

In [ ]:
corr = []
sol = []

for i in corr_filtered_ix:
    corr.append(genfromtxt(storage_dir + "/corr_matrices/corr{0}.csv".format(i), delimiter=','))
    sol.append(genfromtxt(storage_dir + "/est_uniform/est_uniform{0}.csv".format(i), delimiter=','))

In [ ]:
sol = np.array(sol)
corr = np.array(corr)
corr.shape, sol.shape

### Plot the solution

In [ ]:
K = sol.shape[0]


for i in range(0, 2):
    fig, axes = plt.subplots(1, 2, sharex=True, figsize=(30,30))
    
    ax = sns.heatmap(corr[i, :], ax=axes[0], center=0, vmin = -0.5, vmax = 0.5, 
                     square = True, cbar = False, cmap = "coolwarm", xticklabels=False, yticklabels=False)
    ax.set_title('Covariance')
    
    ax = sns.heatmap(sol[i, :], ax=axes[1], center=0, vmin = -0.5, vmax = 0.5,
                     square = True, cbar = False, cmap = "coolwarm", xticklabels=False, yticklabels=False)
    ax.set_title('Inverse Covariance')
    
    fig.show()
    
    #fig.savefig("/storage/groups/bds01/datasets/brains/plots/SGL_plots/SGL_heatmap{0}.png".format(i))

In [ ]:
sex = pd.read_csv(storage_dir + "sex.csv")

In [ ]:
print("The sex of {0} people - unknown (test set)".format(sex["sex_f0_m1"].isna().sum()))

In [ ]:
sex = sex.reset_index()

male_ixs = np.array(sex[sex["sex_f0_m1"] == 1.0]["index"])
female_ixs = np.array(sex[sex["sex_f0_m1"] == 0.0]["index"])

male_ixs.shape, female_ixs.shape

In [ ]:
P_male = calculate_edge_probablity(data=sol, indices=male_ixs)
P_female = calculate_edge_probablity(data=sol, indices=female_ixs)

Set probability threshold = 0.8

In [ ]:
P_male[P_male < 0.8] = 0
P_female[P_female < 0.8] = 0

In [ ]:
plt.figure(figsize=(16,16))
ax = sns.heatmap(P_male, center=0, vmin = -0.5, vmax = 0.5, square = True, cbar = False, 
                 cmap = "coolwarm", xticklabels=False, yticklabels=False)


fig, axes = plt.subplots(1, 2, sharex=True, figsize=(30,30))

ax = sns.heatmap(P_male, ax=axes[0], center=0, vmin = -0.5, vmax = 0.5, 
                 square = True, cbar = False, cmap = "coolwarm", xticklabels=False, yticklabels=False)
ax.set_title("Male's connectivity network")

ax = sns.heatmap(P_female, ax=axes[1], center=0, vmin = -0.5, vmax = 0.5,
                 square = True, cbar = False, cmap = "coolwarm", xticklabels=False, yticklabels=False)
ax.set_title("Female's connectivity network")

fig.show()

fig.savefig("/mnt/home/icb/oleg.vlasovetc/brain_challenge/f-threshold-select/plots/connectivity_heatmap.png")